In [21]:
import os.path
import re
import tika
from tika import parser
import nltk
from algorithms import max_subarray
from nltk.tokenize import PunktSentenceTokenizer,  word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
tika.initVM()

In [22]:
filename = '/home/abhishek/My_resume.pdf'
#filename = 'testing1.docx'
#filename = 'testing_2.pdf'
#filename = 'RoushanRanjan3_0.pdf'
#filename = 'Resume.docx'
parsed = parser.from_file(filename)
content = parsed["content"]
#print(content)
keys = []
with open("word_list.txt") as infile:
    for i in infile:
        i = i.strip()
        keys.append(i)

In [23]:
ps = PorterStemmer()

example_text = "This is a good icecream. It is nice product. I hate coca cola. The weather is great and pyhton is awesome. The sky is blue and i was running. I have completed my b.tech from c.v raman institute in information technology from kalyani on 5/12/2018. Cats are better than dogs"
custom_sent_tokenizer = PunktSentenceTokenizer(content)
tokenized = custom_sent_tokenizer.tokenize(content)

In [24]:
sent = content.split("\n")
for i in sent:
    print(i)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

hash_map = {}
actual_map = {}


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

@@@@@@@@@@

In [25]:
def process_content():
  index = 0
  try:
    for i in sent:
      original_text = i
      i = re.split('[#$%*:”&()<>?;·/\|}{~,]|\s',i.lower())
      i = " ".join(i)
      words = nltk.word_tokenize(i)
      #print(words)
      stop_words = set(stopwords.words("english"))
      filtered_sentence = []
      for w in words:
        if w not in stop_words:
          filtered_sentence.append(w)

      #print(filtered_sentence)

      for ind in range(len(filtered_sentence)-1):
        wo1 = filtered_sentence[ind]
        wo2 = filtered_sentence[ind+1]
        combined_words = wo1 + " " + wo2

        if wo1 in keys:
            hash_map[index] = i
            actual_map[index] = original_text
        elif wo2 in keys:
            hash_map[index] = i
            actual_map[index] = original_text
        elif combined_words in keys:
            hash_map[index] = i
            actual_map[index] = original_text

      index += 1
      #chunked.draw()
  except Exception as e:
    print(str(e))

  return hash_map, actual_map;

In [26]:
hash_map, actual_map = process_content()
#print(hash_map)
text_list = []
line_numbers = []
for i in hash_map:
    text_list.append(hash_map[i])
    line_numbers.append(i)

In [27]:
details, clusterd_ids = [], []
for line_id in (range(len(text_list))):
    i = text_list[line_id]
    words = nltk.word_tokenize(i)
    tagged = nltk.pos_tag(words)
    #print(tagged)
    namedEnt = nltk.ne_chunk(tagged)
    #print(namedEnt)
    #print("........................................")
    chunkGram1 = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?<IN>*<CD>?}"""
    chunkGram2 = r"""Chunk: {<NN.?>+<JJ|IN|CD|DT|VB.?|NN.?>+<NN.?>+| <JJ>*<IN|CD|NN.?|VB.?|JJ>+<NN.?>+}"""
    chunkGram3 = r"""Chunk: {<NN.?>*<CD>+<NN.?>?}"""
    chunkParser3 = nltk.RegexpParser(chunkGram3)
    chunkParser1 = nltk.RegexpParser(chunkGram1)
    chunkParser2 = nltk.RegexpParser(chunkGram2)
    chunked1 = chunkParser1.parse(tagged)

    chunked2 = chunkParser2.parse(tagged)
    chunked3 = chunkParser3.parse(tagged)
    
    ans1 = []
    for info in chunked2:
        if (type(info) != tuple):
            #print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
            ans1 += info[0:]
            
    ans2 = []
    for info in chunked3:
        if (type(info) != tuple):
            #print("*************************************")
            ans2 += info[0:]

    if(ans1 != [] or ans2 != []):
        a1 = []
        a2 = []
        for a in ans1:
            a1.append(a[0])
        for a in ans2:
            a2.append(a[0])

        a1 = " ".join(a1)
        a2 = " ".join(a2)

        temp = {"NN": a1, "NUM": a2, "line_id": line_numbers[line_id]}
        clusterd_ids.append(line_numbers[line_id])
        details.append(temp)

In [28]:
for d in details:

    #print("Name related details : ", d["NN"])
    print("Line number : ", d["line_id"])
    print("Actual line : ", actual_map[d["line_id"]])
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Line number :  77
Actual line :  May - Jul 2017 Summer Intern at IIT Kharagpur
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  87
Actual line :  May 2018 B.Tech in Computer Science
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  88
Actual line :  Institute : Indian Institute of Information Technology, Kalyani
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  90
Actual line :  CGPA: 9.2/10
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  92
Actual line :  May 2014 12th CBSE
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  93
Actual line :  School : Kendriya Vidyalaya, Dipatoli Ranchi
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  95
Actual line :  Percentage: 93.60%
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  97
Actual line :  May 2012 10th CBSE
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Line number :  98
Actual line :  School : Kendriya Vidyalaya, Dipatoli Ranchi
>>>>>>>>>>>>>

In [29]:
clusterd_ids = max_subarray(clusterd_ids)
#print(clusterd_id)
for d in clusterd_ids:
    #print("Line number : ", d)
    print("Actual line : ", actual_map[d])
    #print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Actual line :  May 2018 B.Tech in Computer Science
Actual line :  Institute : Indian Institute of Information Technology, Kalyani
Actual line :  CGPA: 9.2/10
Actual line :  May 2014 12th CBSE
Actual line :  School : Kendriya Vidyalaya, Dipatoli Ranchi
Actual line :  Percentage: 93.60%
Actual line :  May 2012 10th CBSE
Actual line :  School : Kendriya Vidyalaya, Dipatoli Ranchi
Actual line :  CGPA: 9.4/10
Actual line :  Gate - 2018 Marks : 45, Gate - 2018 Score : 588
